In [11]:
import pandas as pd

In [14]:
df = pd.read_csv('./data/full_data.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126662 entries, 0 to 126661
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   num_acc      126662 non-null  int64  
 1   id_usager    126662 non-null  int64  
 2   id_vehicule  126662 non-null  int64  
 3   num_veh      126662 non-null  object 
 4   place        126662 non-null  int64  
 5   catu         126662 non-null  int64  
 6   grav         126662 non-null  int64  
 7   sexe         126662 non-null  int64  
 8   an_nais      126662 non-null  int64  
 9   trajet       126662 non-null  int64  
 10  locp         126662 non-null  int64  
 11  actp         126662 non-null  object 
 12  etatp        126662 non-null  int64  
 13  catv         126662 non-null  int64  
 14  obs          126662 non-null  int64  
 15  obsm         126662 non-null  int64  
 16  choc         126662 non-null  int64  
 17  manv         126662 non-null  int64  
 18  lum          126662 non-